<a href="https://colab.research.google.com/github/tntly/ml-projects/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten Digit Classification with MNIST

This notebook demonstrates a simple implementation of a neural network for handwritten digit classification using the MNIST dataset. It was created as part of my coursework for CS 271: Topics in Machine Learning at San Jose State University.

The MNIST dataset is a widely used benchmark in the field of computer vision, containing 70,000 grayscale images of handwritten digits (0-9), each with a resolution of 28x28 pixels. In this example, I follow a standard pipeline for training a classifier using TensorFlow, which includes:

* Loading and preprocessing the MNIST dataset
* Building and training a neural network model
* Evaluating the model's performance

This notebook serves as a foundational exercise in deep learning and provides insights into image classification using neural networks.

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# Load the data
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# mnist_dataset is a dictionary with two key-value pairs: 'train' and 'test'
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [ ]:
# Define the number of validation samples as a percentage of the training data
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# Cast the number of validation samples to an integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# Define the number of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [ ]:
# Define a function that scales the images
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

# Apply the scale function to the data
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [ ]:
BUFFER_SIZE = 10000
# Shuffle the data
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# Extract the validation and train data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [ ]:
BATCH_SIZE = 100
# Batch the data
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [ ]:
# Define the model
input_size = 784
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer

    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer

    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

# Define the optimizer and the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Define the number of epochs
NUM_EPOCHS = 5
# Train the model
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data, verbose=2)

Epoch 1/5
540/540 - 9s - 16ms/step - accuracy: 0.8899 - loss: 0.3935 - val_accuracy: 0.9437 - val_loss: 0.2047
Epoch 2/5
540/540 - 4s - 7ms/step - accuracy: 0.9490 - loss: 0.1755 - val_accuracy: 0.9573 - val_loss: 0.1483
Epoch 3/5
540/540 - 4s - 7ms/step - accuracy: 0.9613 - loss: 0.1335 - val_accuracy: 0.9650 - val_loss: 0.1158
Epoch 4/5
540/540 - 5s - 9ms/step - accuracy: 0.9682 - loss: 0.1069 - val_accuracy: 0.9712 - val_loss: 0.1011
Epoch 5/5
540/540 - 4s - 7ms/step - accuracy: 0.9728 - loss: 0.0904 - val_accuracy: 0.9682 - val_loss: 0.1052


In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 926ms/step - accuracy: 0.9667 - loss: 0.1062
Test loss: 0.11. Test accuracy: 96.67%
